# Evaluating Numerical Expressions with Stacks

In this project, we would be using stacks to implement an algorithm that can evaluate numerical expressions, even complex ones. A function named `evaluate` would be used to evaluate expressions stored in a string. An implementation of a Linked List has been created beforehand and is contained within the `linked_list.py` file. The stack implementation will be based on the Linked List implementation.

In [1]:
from linked_list import LinkedList

In [2]:
class Stack(LinkedList):
    def push(self, data):
        self.append(data)
    def peek(self):
        return self.tail.data
    def pop(self):
        ret = self.tail.data
        if self.length == 1:
            self.tail = self.head = None
        else:
            self.tail = self.tail.prev
            self.tail.next = None
        self.length -= 1
        return ret

# Infix and Postfix Notation

A computer can easily evaluate an expression written in **postfix notation**, where the operands appear before the operator. For example, the infix notation for the expression is `1 + 2` which is converted to `1 2 +` in postfix notation. We can evaluate an expression in postfix notation using a stack. We read the expression from left to right and do the following:
1. If we find a number, we push that number to the top of the stack.
2. If we find an operator, we pop the top two elements of the stack, perform the operation, and then push back the result.

It is noteworthy that when we perform the operation, the top element of the two popped would be the second operand and the bottom would be first operand. The operator must be applied in the correct order to the two popped operands from the stack in the second step.

After processing the entire expression, there will be a single element on the stack. This value is the result of the operation.

# Tokenization

Let's implement a function `evaluate_postfix` that would evaluate an expression in postfix notation which is in a form of a string. We will assume that the string passed in would have spaces between all elements of the expression. With this assumption, we can implement a `tokenize` function that would convert the expression which is a string to a list of strings of which the elements (tokens) are within this list.

In [3]:
# Tokenization function to transform an expression into a list of tokens
def tokenize(string):
    return string.split(" ")

In [4]:
# Testing out tokenize function
tokenize("1 2 +")

['1', '2', '+']

# Processing an Operator

In [5]:
# Function that processes the `-` token
def process_minus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top - top
    stack.push(result)

In [6]:
# Function that processes the `+` token
def process_plus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top + top
    stack.push(result)

In [7]:
# Function that processes the `*` token
def process_times(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top * top
    stack.push(result)

In [8]:
# Function that processes the `/` token
def process_divide(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top / top
    stack.push(result)

In [9]:
# Function that processes the `**` token
def process_pow(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top ** top
    stack.push(result)

# Evaluting Postfix Expressions

In [10]:
# Implementing the function that can evaluate a postfix expression

def evaluate_postfix(expression):
    tokenized = tokenize(expression)
    stack = Stack()
    for token in tokenized:
        if token == "+":
            process_plus(stack)
        elif token == "-":
            process_minus(stack)
        elif token == "*":
            process_times(stack)
        elif token == "/":
            process_divide(stack)
        elif token == "**":
            process_pow(stack)
        else:
            stack.push(float(token))
    return stack.peek()

In [11]:
# Testing out the function
expressions = [
    "4 6 -",
    "4 1 2 9 3 / * + 5 - *",
    "1 2 + 3 -",
    "1 2 - 3 +",
    "10 3 5 * 16 4 - / +",
    "5 3 4 2 - ** *",
    "12 2 4 + / 21 *",
    "1 1 + 2 **",
    "1 1 2 ** +"
]

In [12]:
results = []
for exp in expressions:
    results.append(evaluate_postfix(exp))
results

[-2.0, 8.0, 0.0, 2.0, 11.25, 45.0, 42.0, 4.0, 2.0]

Seems to work alright!

# Operator Precedence in Infix Notation

Now we can evaluate postfix expressions but most of the time, we are interested in evaluating expressions in infix notation. After all, it would be very annoying to have to write expressions in postfix notation in order to use our algorithm as implemented above. To convert out expression from infix to postfix notation, the Shunting-yard algorithm has to be implemented, which can be implemented using a stack data structure. In the Shunting-yard algorithm, we need to compare the precedence of the operators. We will use numbers to represent the operator precedence. The higher the precedence, the higher the number.

In [13]:
# Dictionary to store the operator precedence
precedence = {
    "+" : 1,
    "-" : 1,
    "*" : 2,
    "/" : 2,
    "**": 3,
}

In [14]:
# test to see if precedence of `/` is smaller than `-`
precedence["/"] < precedence["-"]

False

# From Infix to Postfix

The `infix_to_postfix` function, when given a string with an infix expressions, will output a string with that expression written in postfix notation. This function would implement the Shunting-yard algorithm.

## Handling Opening Parenthesis

In [15]:
# Process opening parenthesis
def process_opening_parenthesis(stack):
    stack.push("(")

## Handling Closing Parenthesis

In [16]:
# Process closing parenthesis
def process_closing_parenthesis(stack, postfix):
    while stack.peek() != "(":
        postfix.append(stack.pop())
    stack.pop()

## Handling Operators

In [17]:
# Process operators
def process_operator(stack, postfix, operator):
    while len(stack) > 0 and stack.peek() in precedence and precedence[stack.peek()] >= precedence[operator]:
        postfix.append(stack.pop())
    stack.push(operator)

## Handling Numbers

In [18]:
# Process numbers
def process_number(postfix, number):
    postfix.append(number)

## Putting it all together

We now have all what we need to implement the `infix_to_postfix` function that uses the shunting-yard algorithm. This function will work as follows:

1. We start by splitting the expression into tokens using the tokenize() function.
2. We initialize an empty stack.
3. We initialize an empty postfix token list.
4. Iterate over all tokens, and for each, do the following:
    * If the token is "(", we call the process_opening_parenthesis() function.
    * If the token is ")", we call the process_closing_parenthesis() function.
    * If the token is an operator, we call the process_operator() function.
    * Otherwise, the token is a number, and we call the process_number() function.
5. After processing all tokens, we use a while loop to pop the remaining stack element into the postfix token list.
6. Use the str.join() method to convert the postfix token list into a string.

In [19]:
# Implementing the function to convert infix to postfix expressions
def infix_to_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    postfix = []
    for token in tokens:
        if token == "(":
            process_opening_parenthesis(stack)
        elif token == ")":
            process_closing_parenthesis(stack, postfix)
        elif token in ("+", "-", "/", "*", "**"):
            process_operator(stack, postfix, token)
        else:
            process_number(postfix, token)
    while len(stack) > 0:
        postfix.append(stack.pop())
    return " ".join(postfix)

With that, we can combine the previous `evaluate_postfix` together with the `infix_to_postfix` functions to put together an `evaluate` function that can return the value of an expression in infix notation.

In [20]:
# Putting all together for the evaluate function to evaluate an infix expression
def evaluate(expression):
    postfix_expression = infix_to_postfix(expression)
    result = evaluate_postfix(postfix_expression)
    return result

In [21]:
# Testing out
expressions = [
    "1 + 1",
    "1 * ( 2 - ( 1 + 1 ) )",
    "4 * ( 1 + 2 * ( 9 / 3 ) - 5 )",
    "10 + 3 * 5 / ( 16 - 4 * 1 )",
    "2 * 2 * 2 * 2 * 2 * 2 * 2 * 2",
    "2 ** 2 ** 2 ** 2 ** 2",
    "( 1 - 2 ) / ( 3 - 5 )",
    "9 / 8 * 8",
    "64 / ( 8 * 8 )",
]

results = [evaluate(exp) for exp in expressions]

In [22]:
results

[2.0, 0.0, 8.0, 11.25, 256.0, 65536.0, 0.5, 9.0, 1.0]

All is working fine! Our implementation to evaluate numerical expressions is done.